In [ ]:
!pip install langchain-community langchain langchain-openai vllm chromadb

In [2]:
from langchain_community.llms import VLLM

llm = VLLM(model="bigscience/bloom-3b", temperature=0.7, max_new_tokens=128, top_p=0.96, top_k=20, trust_remote_code=True)

WARNING 01-19 18:38:42 cuda.py:32] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.
INFO 01-19 18:38:46 config.py:2272] Downcasting torch.float32 to torch.float16.
WARNING 01-19 18:38:46 config.py:2339] The model's config.json does not contain any of the following keys to determine the original maximum length of the model: ['max_position_embeddings', 'n_positions', 'max_seq_len', 'seq_length', 'model_max_length', 'max_sequence_length', 'max_seq_length', 'seq_len']. Assuming the model's maximum length is 2048.
INFO 01-19 18:38:56 config.py:510] This model supports multiple tasks: {'classify', 'generate', 'embed', 'reward', 'score'}. Defaulting to 'generate'.
INFO 01-19 18:38:56 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='bigscience/bloom-3b', 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 01-19 18:39:35 model_runner.py:1099] Loading model weights took 5.6083 GB
INFO 01-19 18:39:37 worker.py:241] Memory profiling takes 2.17 seconds
INFO 01-19 18:39:37 worker.py:241] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 01-19 18:39:37 worker.py:241] model weights take 5.61GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 2.29GiB; the rest of the memory reserved for KV Cache is 5.28GiB.
INFO 01-19 18:39:37 gpu_executor.py:76] # GPU blocks: 1153, # CPU blocks: 873
INFO 01-19 18:39:37 gpu_executor.py:80] Maximum concurrency for 2048 tokens per request: 9.01x
INFO 01-19 18:39:43 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:31<00:00,  1.10it/s]

INFO 01-19 18:40:15 model_runner.py:1535] Graph capturing finished in 32 secs, took 0.23 GiB
INFO 01-19 18:40:15 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 40.22 seconds


In [3]:
print(llm.invoke("Who wrote Rhapsody in Blue?"))

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.72s/it, est. speed input: 1.69 toks/s, output: 27.10 toks/s]

"

"No, I did not; I am sure I did not."

"Who wrote it?"

"I do not know."

"Why do you say you do not know?"

"I do not know."

"How do you know?"

"I do not know."

"Why do you say you do not know?"

"I do not know."

"How do you know?"

"I do not know."

"Why do you say you do not know?"

"I do not know."

"How do you know?"

"I do


In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
import chromadb
from uuid import uuid4
from langchain.llms import OpenAI

In [7]:
folder_path = "/kaggle/input/book-dataset/book.txt"
loader = TextLoader(folder_path)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200
)

split_docs = splitter.split_documents(documents)
embeddings = SentenceTransformerEmbeddings(model_name="static-retrieval-mrl-en-v1")
collection = Chroma.from_documents(split_docs, embeddings, persist_directory='chroma_db')

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the question based on the book 'Perspectives in Logic by Stephen Cook and Phuong Nguyen'. "
        "Context: {context}"
        "Question: {input}"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
pipeline = create_retrieval_chain(collection.as_retriever(), question_answer_chain)

In [11]:
resp = pipeline.invoke({"input" : "Give definition of Two-Sorted First-Order Vocabularies"})
print(resp['answer'])

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.49s/it, est. speed input: 159.00 toks/s, output: 23.31 toks/s]


IV.2. Two-Sorted First-Order Logic
IV.2.1. Syntax. Our two-sorted ﬁrst-order logic is an extension of the
(single-sorted) ﬁrst-order logic introduced in Chapter II. Here there are
two kinds of variables: the variables x, y, z, . . . of the ﬁrst sort are called
number variables, and are intended to range over the natural numbers; and
the variables X, Y, Z, . . . of the second sort are called set (or also string)
variables, and are intended to range over ﬁnite subsets of natural numbers
